<a href="https://colab.research.google.com/github/tarunchand/Big-Data/blob/main/BigDataLab_28_10_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install pyspark
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=baf7442dba175451b0daa9ce0d2218afafb5df938287c2c35a373c73fd02cb88
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
# Linking with Spark
from pyspark import SparkContext, SparkConf

# Initializing Spark
conf = SparkConf().setAppName("test").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [ ]:
sc.defaultParallelism

2

In [ ]:
import random
data = random.sample(range(0, 50), 20)
data

[33, 6, 36, 8, 34, 46, 27, 30, 38, 35, 13, 18, 11, 42, 39, 49, 7, 41, 26, 17]

In [ ]:
rdd = sc.parallelize(data, 4)

In [ ]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
rdd.collect()

[33, 6, 36, 8, 34, 46, 27, 30, 38, 35, 13, 18, 11, 42, 39, 49, 7, 41, 26, 17]

In [ ]:
rdd.take(5)

[33, 6, 36, 8, 34]

In [ ]:
rdd.takeOrdered(5)

[6, 7, 8, 11, 13]

In [ ]:
rdd.glom().collect()

[[33, 6, 36, 8, 34],
 [46, 27, 30, 38, 35],
 [13, 18, 11, 42, 39],
 [49, 7, 41, 26, 17]]

In [ ]:
# map(func)
def my_fun(x):
  return (x + 2) / 2

rdd1 = rdd.map(my_fun)
rdd1.glom().collect()

[[17.5, 4.0, 19.0, 5.0, 18.0],
 [24.0, 14.5, 16.0, 20.0, 18.5],
 [7.5, 10.0, 6.5, 22.0, 20.5],
 [25.5, 4.5, 21.5, 14.0, 9.5]]

In [ ]:
# map(func)
rdd1 = rdd.map(lambda x:(x + 3) / 7)
rdd1.glom().collect()

[[5.142857142857143,
  1.2857142857142858,
  5.571428571428571,
  1.5714285714285714,
  5.285714285714286],
 [7.0,
  4.285714285714286,
  4.714285714285714,
  5.857142857142857,
  5.428571428571429],
 [2.2857142857142856, 3.0, 2.0, 6.428571428571429, 6.0],
 [7.428571428571429,
  1.4285714285714286,
  6.285714285714286,
  4.142857142857143,
  2.857142857142857]]

In [ ]:
rdd3 = rdd.filter(lambda x : x%3 == 0)
rdd3.glom().collect()

[[33, 6, 36], [27, 30], [18, 42, 39], []]

In [ ]:
rdd3.reduce(lambda a,b : a + b)


231

In [ ]:
rdd4 = sc.parallelize([(1,2),(4,5),(1,7),(4,2),(5,3),(4,9)])
rdd4.collect()

[(1, 2), (4, 5), (1, 7), (4, 2), (5, 3), (4, 9)]

In [ ]:
# reduceByKey(func)
rdd5 = rdd4.reduceByKey(lambda a,b: a+b)
rdd5.collect()

[(4, 16), (1, 9), (5, 3)]

In [ ]:
rdd5.sortByKey().collect()

[(1, 9), (4, 16), (5, 3)]

In [ ]:
print([rdd1.max(), rdd1.min(), rdd1.mean(), rdd1.sum()])

[7.428571428571429, 1.2857142857142858, 4.4, 88.0]


In [ ]:
# union():
data1 = random.sample(range(0, 20), 10)
data2 = random.sample(range(0, 20), 10)

rdd1 = sc.parallelize(data1,2)
rdd2 = sc.parallelize(data2,2)
rdd_union = rdd1.union(rdd2)

print("rdd1: ", rdd1.glom().collect())
print("rdd2: ", rdd2.glom().collect())

print('new partition: ', rdd_union.getNumPartitions()) # print the number of Partitions
print(rdd_union.glom().collect())

rdd1:  [[9, 15, 7, 11, 12], [19, 17, 18, 2, 4]]
rdd2:  [[8, 1, 0, 5, 6], [2, 14, 4, 12, 3]]
new partition:  4
[[9, 15, 7, 11, 12], [19, 17, 18, 2, 4], [8, 1, 0, 5, 6], [2, 14, 4, 12, 3]]


In [ ]:
# intersection():
rdd_intersection = rdd1.intersection(rdd2)

print(rdd_intersection.getNumPartitions())
print(rdd_intersection.collect())
print(rdd_intersection.glom().collect())

4
[12, 4, 2]
[[12, 4], [], [2], []]


In [ ]:
rdd_intersection.coalesce(1).glom().collect()

[[12, 4, 2]]